420-A52-SF - Algorithmes d'apprentissage supervisé - Hiver 2020 - Spécialisation technique en Intelligence Artificielle - Mikaël Swawola, M.Sc.
<br/>
![Correction Projet #1](static/projet1-banner.png)

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## 0 - Chargement des bibliothèques

In [ ]:
# Manipulation de données
import numpy as np
import pandas as pd

# Visualisation de données
import matplotlib.pyplot as plt
import seaborn as sns

# Helpers
from helpers import polynomial

# Outils divers
import time
from collections import defaultdict

# Machine Learning
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [ ]:
# Configuration de la visualisation
sns.set(style="darkgrid", rc={'figure.figsize':(12,6)})
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

## 1 - Chargement du jeu de données et exploration sommaire

<strong>Lecture du fichier `BOS_train.csv`<strong/>

In [ ]:
BOS = pd.read_csv('BOS_train.csv', index_col=[0])

<strong>Affichage des dix premières lignes de la trame de données BOS</strong>

In [ ]:
BOS.head()

<strong>Affichage du nombre d'observations</strong>

In [ ]:
len(BOS)

#### Affichage du nom des variables indicatrices

In [ ]:
BOS.columns

| Variable        | Description           | Type  |
| --------------- |:--------------------- |:----- |
| origin      | Aéroport de départ | qualitative |
| time_hour      | Date et heure prévues du vol      |   qualitative |
| tailnum | Numéro d'immatriculation      |    qualitative |
| year | Année de départ | quantitative |
| month | Mois de départ | quantitative | 
| day | Jour de départ | quantitative |
| dep_time | Heure de départ effective | quantitative |
| sched_dep_time | Heure de départ prévue | quantitative |
| dep_delay | Retard au départ (en minutes). Les temps négatifs représentent une avance | quantitative |
| **arr_delay** | **Retard à l'arrivée (en minutes). Les temps négatifs représentent une avance** | **quantitative** |
| carrier | Transporteur (abbréviation) |    qualitative |
| flight | Numéro du vol |    qualitative |
| dest | Aéroport de destination |    qualitative |
| air_time | Temps passé dans les airs (en minutes)  | quantitative |
| distance | Distance entre les aéroports (en miles) | quantitative |
| hour | Heure de départ prévue | quantitative |
| minute | Minute de départ prévue | quantitative |
| seats | Nombre de sièges dans l'avion | quantitative |
| temp | Température en F | quantitative |
| dewp | Point de rosée en F | quantitative |
| humid | Humidité relative | quantitative |
| wind_dir |  Direction du vent (en degrés) | quantitative |
| wind_speed | Vitesse du vent (en mph) | quantitative |
| precip | Preciptations (en pouces) | quantitative |
| pressure | Pression au niveau de la mer (en millibars) | quantitative |
| visib | Visibilité (en miles) | quantitative |
| week_day | jour de la semaine du départ du vol | qualitative |
| wknd | indicatrice de la fin de semaine (TRUE pour un vol de fin de semaine, FALSE sinon) | qualitative |
| evening_rush_hour | indicatrice de l'heure de pointe de fin de journée (TRUE entre 17 et 19 heures du soir, FALSE sinon) | qualitative |
| wind_sin | variable pour modéliser correctement les vents (wind_speed x sin(BOS\$wind_dir x pi/180)) | quantitative |
| wind_cos | variable pour modéliser correctement les vents (wind_speed x cos(BOS\$wind_dir x pi/180)) | quantitative |
| precip_indic | indicatrice de la présence de précipitation (TRUE s'il y a des précipitations, FALSE sinon) | qualitative |

## 2 - Modèle de référence

Nous remarquons la colonne `dep_delay` (retard au départ). Il est fort probable qu'une relation linéaire existe entre `dep_delay` et `arr_delay`. Affichons le nuage de points correspondant

In [ ]:
g = sns.scatterplot(x='dep_delay', y='arr_delay', data=BOS)
g.set_xlabel("Retard au départ")
g.set_ylabel("Retard à l'arrivée")

In [ ]:
X_base = BOS['dep_delay'].values.reshape(-1,1)
y = BOS['arr_delay'].values.reshape(-1,1)

Effectuons une régression linéaire

In [ ]:
lr_baseline = LinearRegression().fit(X_base, y)

In [ ]:
print(f'{lr_baseline.intercept_}')
print(f'{lr_baseline.coef_}')

In [ ]:
g = sns.scatterplot(x='dep_delay', y='arr_delay', data=BOS)
g.set_xlabel("Retard au départ")
g.set_ylabel("Retard à l'arrivée")

x_grid = np.linspace(X_base.min(), X_base.max(), 10).reshape(-1,1)
y_grid = lr_baseline.predict(x_grid)

plt.plot(x_grid, y_grid, color="g")

### R2

In [ ]:
lr_baseline.score(X_base, y)

### RMSE

In [ ]:
np.sqrt(mean_squared_error(y, lr_baseline.predict(X_base)))

Ce modèle simpliste se trompe en moyenne de 16.05 minutes sur les prévisions de retard à l'arrivée. Nous allons tenter d'améliorer cette performance en ajoutant d'autres variables ayant une influence sur `arr_delay`

# 3 - Régression linéaire

Nous allons maintenant inclure de nouvelles variables dans le modèle. Bien sûr, n'oublions pas d'ajouter celle qui semble la plus significative, `dep_delay`

In [ ]:
df = pd.DataFrame()

## dep_delay

In [ ]:
df['dep_delay'] = BOS['dep_delay']

## dest

`dest` est une variable explicative ne contenant qu'une seule valeur, "BOS". C'est normal pusique le jeu de données ne concerne que les vols en destination de Boston. **Nous pouvonc donc ignorer cette variable**

In [ ]:
BOS['dest'].unique()

## origin

`origin` est une variable qualitative. Voyons les différentes valeurs

In [ ]:
BOS['origin'].unique()

Puisque nous avons 3 valeurs (EWR, JFK et LGA), une première approche consiste à créer 3-1=2 variables indicatrices. Or, si nous regardons le jeu de données attentivement, nous constatons la présence d'une variable `distance` représentant le distance entre les aéroports

In [ ]:
BOS['distance'].unique()

L'information contenue dans ces deux colonnes est donc redondante. Puisque la colonne distance est une variable explicative continue, celle-ci contient plus d'information que la variable explicative `origin`. Une autre raison et que nous n'avons qu'une seule variable au lieu de deux. Nous allons inclure `distance`.

In [ ]:
df['distance'] = BOS['distance']

In [ ]:
lr = LinearRegression().fit(df.values, y)
lr.score(df.values, y)

## time_hour, year, month, day, sched_dep_time, hour, minute, dep_time, dep_delay

In [ ]:
BOS[['time_hour', 'year','month', 'day', 'sched_dep_time','hour','minute','dep_time','dep_delay']].head(10)

Nous pouvons remarquer que:

* La totalité de l'information contenue dans `time_hour` est présente dans les autres variables explicatives
* `year` et `month` ne contiennent respectivement que les valeurs 2013 et 7
* `dep_delay = dep_time - sched_dep_time`
* La totalité de l'information contenue dans `sched_dep_time` est présente dans `hour` et `minute`


In [ ]:
df['day'] = BOS['day']
df['time'] = 60 * BOS['hour'] + BOS['minute']

In [ ]:
lr = LinearRegression().fit(df.values, y)
lr.score(df.values, y)

## week_day

Nous gardons `week_day`, car nous suspectons que le jour de la semaine peut avoir un impact. Nous convertissons en variable continue

In [ ]:
BOS['week_day']

In [ ]:
df['week_day'] = BOS['week_day'].apply(lambda x: time.strptime(x, "%A").tm_wday)
df['week_day'].unique()

In [ ]:
lr = LinearRegression().fit(df.values, y)
lr.score(df.values, y)

## wknd, evening_rush_hour

In [ ]:
 BOS[['wknd','evening_rush_hour']].head()

In [ ]:
df['wknd'] = BOS['wknd'].astype(int)
df['evening_rush_hour'] = BOS['evening_rush_hour'].astype(int)

In [ ]:
lr = LinearRegression().fit(df.values, y)
lr.score(df.values, y)

## tailnum, flight

In [ ]:
len(BOS['tailnum'].unique())

In [ ]:
BOS['flight'].unique()

On laisse de côté ces deux variables qualitatives. De toute manière l'intuition indique que celles-ci ne devraient pas être significatives

## carrier

Il est très probable que cette variable montre un impact

In [ ]:
BOS['carrier'].unique()

On crée donc 7-1 variables indicatrices

In [ ]:
df['carrier'] = BOS['carrier']

In [ ]:
df = pd.get_dummies(df, prefix="carrier", drop_first=True)

In [ ]:
lr = LinearRegression().fit(df.values, y)
lr.score(df.values, y)

## air_time

In [ ]:
df['air_time'] = BOS['air_time']

## seats

In [ ]:
df['seats'] = BOS['seats']

In [ ]:
lr = LinearRegression().fit(df.values, y)
lr.score(df.values, y)

In [ ]:
np.sqrt(mean_squared_error(y, lr.predict(df.values)))

## temp, dewp, humid, pressure

In [ ]:
BOS[['temp','dewp','humid','pressure']]

In [ ]:
df['temp'] = BOS['temp']
df['dewp'] = BOS['dewp']
df['humid'] = BOS['humid']
df['pressure'] = BOS['pressure']

In [ ]:
lr = LinearRegression().fit(df.values, y)
lr.score(df.values, y)

## wind_dir, wind_speed, wind_sin, wind_cos

| Variable        | Description           | Type  |
| --------------- |:--------------------- |:----- |
| wind_dir |  Direction du vent (en degrés) | quantitative |
| wind_speed | Vitesse du vent (en mph) | quantitative |
| wind_sin | variable pour modéliser correctement les vents (wind_speed x sin(BOS\$wind_dir x pi/180)) | quantitative |
| wind_cos | variable pour modéliser correctement les vents (wind_speed x cos(BOS\$wind_dir x pi/180)) | quantitative |

In [ ]:
df['wind_sin'] = BOS['wind_sin']
df['wind_cos'] = BOS['wind_cos']

In [ ]:
lr = LinearRegression().fit(df.values, y)
lr.score(df.values, y)

## precip, visib

In [ ]:
df['precip'] = BOS['precip']
df['precip_indic'] = BOS['precip_indic'].astype(int)
df['visib'] = BOS['visib']

In [ ]:
lr = LinearRegression().fit(df.values, y)
lr.score(df.values, y)

In [ ]:
np.sqrt(mean_squared_error(y, lr.predict(df.values)))

In [ ]:
len(df.columns)

In [ ]:
lr_best = lr

# 4 - Régression polynomiale

In [ ]:
def LinearRegressionCV(X_train, y_train, n_splits=5):
    h = defaultdict(list)
    
    kf = KFold(n_splits, shuffle=True)

    for train_index, val_index in kf.split(X_train):
        # Préparation des plis
        x_cv_train, x_cv_val = X_train[train_index], X_train[val_index]
        y_cv_train, y_cv_val = y_train[train_index], y_train[val_index]

        # Régression logistique
        lr = LinearRegression(fit_intercept=False).fit(x_cv_train, y_cv_train)

        # Performances par plis - RMSE
        y_pred_train = lr.predict(x_cv_train)
        y_pred_val = lr.predict(x_cv_val)
        h['train'].append(mean_squared_error(y_cv_train, y_pred_train, squared=False))
        h['val'].append(mean_squared_error(y_cv_val, y_pred_val, squared=False))
    
    return np.mean(h['train']), np.mean(h['val'])


history = defaultdict(list)
for n in range(1, 5):
    
    # variables polynomiales
    X_poly = polynomial(df.values, degree=n)
    t, v = LinearRegressionCV(X_poly, y)
    history['train'].append(t)
    history['val'].append(v)

In [ ]:
f, ax = plt.subplots(1,1)
ax.plot(range(1, 5), history['train'], label="train")
ax.plot(range(1, 5), history['val'], label="val")
ax.set_xlabel('n', fontsize=14)
ax.set_ylabel('RMSE', fontsize=14)
ax.legend()

Si on se fie à la courbe ci-dessus, une régression polynomiale n'est pas utile au problème

# 5 - Régression KNN

In [ ]:
def kNNCV(X_train, y_train, n_neighbors, n_splits=5):
    h = defaultdict(list)
    
    kf = KFold(n_splits, shuffle=True)
    
    for train_index, val_index in kf.split(X_train):
        # Préparation des plis
        x_cv_train, x_cv_val = X_train[train_index], X_train[val_index]
        y_cv_train, y_cv_val = y_train[train_index], y_train[val_index]

        # Régression logistique
        neigh = KNeighborsRegressor(n_neighbors=n_neighbors).fit(x_cv_train, y_cv_train)

        # Record performances par plis
        # MSE
        y_pred_train = neigh.predict(x_cv_train)
        y_pred_val = neigh.predict(x_cv_val)
        h['train'].append(np.sqrt(mean_squared_error(y_cv_train, y_pred_train)))
        h['val'].append(np.sqrt(mean_squared_error(y_cv_val, y_pred_val)))
        
    return np.mean(h['train']), np.mean(h['val'])


history = defaultdict(list)
for n_neighbors in range(1, 10):
    
    tr, val = kNNCV(df.values, y, n_neighbors)
    history['train'].append(tr)
    history['val'].append(val)

In [ ]:
f, ax = plt.subplots(1,1)
ax.plot(range(1, 10), history['train'], label="train")
ax.plot(range(1, 10), history['val'], label="val")
ax.set_xlabel('k', fontsize=14)
ax.set_ylabel('RMSE', fontsize=14)
ax.legend()

Comme on peut le constater, les performances de KNN sont très en deça des performances de la régression linéaire

# 6 - Prédictions sur le jeu de test

In [ ]:
def prepare_data(df):
    """
    Préparation et sélection manuelle des variables explicatives
    """
    
    dfp = pd.DataFrame()
    
    dfp['dep_delay'] = df['dep_delay']
    
    dfp['distance'] = df['distance']
    dfp['day'] = df['day']
    dfp['time'] = 60 * df['hour'] + df['minute']
    
    dfp['week_day'] = df['week_day'].apply(lambda x: time.strptime(x, "%A").tm_wday)
    dfp['wknd'] = df['wknd'].astype(int)
    dfp['evening_rush_hour'] = df['evening_rush_hour'].astype(int)
        
    dfp['carrier'] = df['carrier']
    dfp['carrier_AA'] = 0 # AA absent du jeu de test. Attention à l'ordre des colonnes
    dfp = pd.get_dummies(dfp, prefix="carrier", drop_first=True)
    
    
    dfp['air_time'] = df['air_time']
    dfp['seats'] = df['seats']
    
    dfp['temp'] = df['temp']
    dfp['dewp'] = df['dewp']
    dfp['humid'] = df['humid']
    dfp['pressure'] = df['pressure']
    
    
    dfp['wind_sin'] = df['wind_sin']
    dfp['wind_cos'] = df['wind_cos']
    
    dfp['precip'] = df['precip']
    dfp['precip_indic'] = df['precip_indic'].astype(int)
    dfp['visib'] = df['visib']
            
    assert dfp.columns.all(df.columns)
    
    return dfp

In [ ]:
BOS_test = pd.read_csv('BOS_test.csv', index_col=[0])

In [ ]:
BOS_test.head()

In [ ]:
BOS_test.columns

In [ ]:
df_test = prepare_data(BOS_test)

In [ ]:
df_test.head()

In [ ]:
X_test = df_test.values
y_pred_test = lr_best.predict(X_test)
BOS_test['y_pred_test'] = y_pred_test

In [ ]:
BOS_test.to_csv('output.csv', columns=['y_pred_test'])

# 7 - Résultats finaux

In [ ]:
BOS_test_arr_delay = pd.read_csv("BOS_test_arr_delay.csv")

### Meilleur modèle

In [ ]:
y_test = BOS_test_arr_delay['arr_delay'].values

In [ ]:
mean_squared_error(y_pred_test, y_test, squared=False)

### mean baseline

In [ ]:
y_mean_baseline = BOS['arr_delay'].mean()
y_mean_baseline = np.full(shape=(len(y_test),1), fill_value=y_mean_baseline)

In [ ]:
mean_squared_error(y_mean_baseline, y_test, squared=False)

### dep_delay_baseline

In [ ]:
y_base = lr_baseline.predict(df_test[['dep_delay']].values.reshape(-1,1))

In [ ]:
mean_squared_error(y_base, y_test, squared=False)

In [ ]:
16.57-12.25

Trafic à l'Aéroport Montréal-Trudeau en 2018, nombre de vols = 264 195

In [ ]:
264195*4.32/60/24/365.25